In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yfinance as yf
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Float,
    DateTime,
    Integer
)
import sqlite3

## Create a helper function to insert data
`yfinance` has evolved to be able to download multiple tickers at a time but it becomes a nested multi-index dataframe. I prefer to download data for different tickers individually.

In [3]:
def insert_data(
    db_path: str,
    ticker: str,
    date_start: str,
    date_end: str,
) -> None:
    """Function to insert data into a SQLite  database."""
    
    try:
        data = yf.download("ILMN", start=date_start, end=date_end, group_by="ticker")
    except Exception as e:
        raise e
    data = data.xs("ILMN", axis=1, level="Ticker")
    data = data.reset_index()
    engine = create_engine(f"sqlite:///{db_path}")
    metadata_obj = MetaData()
    table = Table(
        ticker.lower(),
        metadata_obj,
        Column("id", Integer, primary_key=True),
        Column("date", DateTime),
        Column("open", Float),
        Column("high", Float),
        Column("low", Float),
        Column("close", Float),
        Column("volume", Integer),
    )
    metadata_obj.create_all(engine)
    conn = sqlite3.connect(db_path)
    data.to_sql(
        ticker.lower(),
        conn,
        if_exists='append',
        index=False
    )
    print(f"Ticker {ticker} added to database")

In [4]:
from datetime import datetime
from tqdm.notebook import tqdm

today = datetime.today().strftime("%Y-%m-%d")

In [5]:
import os

if not os.path.exists("../database"):
    os.makedirs("../database")

In [6]:
for ticker in tqdm(["ILMN", "AAPL", "NVDA"]):
    insert_data(
        db_path = "../database/stocks.db",
        ticker=ticker,
        date_start="2020-01-01",
        date_end=today
    )

  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_1539238/918984288.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("ILMN", start=date_start, end=date_end, group_by="ticker")
[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_1539238/918984288.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("ILMN", start=date_start, end=date_end, group_by="ticker")
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_1539238/918984288.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("ILMN", start=date_start, end=date_end, group_by="ticker")
[*********************100%***********************]  1 of 1 completed

Ticker ILMN added to database
Ticker AAPL added to database
Ticker NVDA added to database


And now we've ingested the data into our database!